# 1. scraping data from the website with scrolling
1-1. scoll to the bottom of best selling product page and scrape product id from products \
1-2. scraping review data from product page 

## 1-1. scoll and save product id from products listed in the order of best selling product 

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import re
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# wait until available to scroll
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys

In [33]:
# Selenium setup
path =  r"C:\Users\z1one\OneDrive\바탕 화면\chromedriver-win64\chromedriver.exe" # Update with the path to your ChromeDriver executable
s = Service(path)
driver = webdriver.Chrome(service=s)
driver.maximize_window()

# Amazon 11st web page URL
url = "https://amazon.11st.co.kr/amazon/category?dispCtgr3No=1150107"
driver.get(url)
driver.implicitly_wait(10) 

#스크롤 전의 스크롤바 높이
before_h=driver.execute_script("return window.scrollY")

while True :
    driver.find_element(by=By.CSS_SELECTOR, value="body").send_keys(Keys.END) #맨 아래로 스크롤 내린다
    time.sleep(2) #스크롤 하는 동안의 로딩시간
    
    
    #스크롤 후의 스크롤바 높이
    after_h=driver.execute_script("return window.scrollY")
    
    if after_h==before_h:
        break
    before_h=after_h

In [34]:
# Function to scroll to the end of the page
def scroll_to_end(driver):
    driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
    time.sleep(2)  #

# Selenium 
path =  r"C:\Users\z1one\OneDrive\바탕 화면\chromedriver-win64\chromedriver.exe"
s = Service(path)
driver = webdriver.Chrome(service=s)
driver.maximize_window()

# Amazon 11st URL
url = "https://amazon.11st.co.kr/amazon/category?dispCtgr3No=1150107"
driver.get(url)
driver.implicitly_wait(10)

# scroll bar heigth before scroll     
before_h = driver.execute_script("return window.scrollY")

# list to save product numbers
product_numbers = []

while len(product_numbers) < 10000:  # set maximum number of products
    # scroll_to_end
    scroll_to_end(driver)

    # scroll bar heigth after scroll     
    after_h = driver.execute_script("return window.scrollY")

    # if the scorll bar doesn't move stop
    if after_h == before_h:
        break

    before_h = after_h

    # Get the current page source
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Find product numbers and add them to the list
    a_tags = soup.find_all('a')
    for a_tag in a_tags:
        href = a_tag.get('href')
        if href and "products/" in href:
            product_number = href.split("products/")[1]
            product_numbers.append(product_number)

# Close the browser
driver.quit()

# Save 1000 product numbers each
file_counter = 1
for i in range(0, len(product_numbers), 1000):
    batch = product_numbers[i:i + 1000]
    output_file = f"product_numbers_{file_counter}.txt" # set directory to save
    with open(output_file, "w") as file:
        file.write("\n".join(batch))
    file_counter += 1

## 1-2. scraping review data from product page 

# 수정

In [2]:
# directory
data_dir = 'data'
output_dir = 'output'
product_numbers_files = [f"{data_dir}/product_numbers_{i}.txt" for i in range(1, 2)]

def scrape_and_save_reviews(product_numbers, output_filename):
    review_cnt = 0
    result = []
    review_product_num_list = []
    path =  r"C:\Users\z1one\OneDrive\바탕 화면\chromedriver-win64\chromedriver.exe"
    s = Service(path)
    # create option to hide widow
    options = webdriver.ChromeOptions()
    options.add_argument("headless")

    driver = webdriver.Chrome(service=s, options  =options)

    for num in product_numbers: # num = productnum
        base_url =f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo=1&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
        req_parmas = {
                "referer":f"https://www.11st.co.kr/products/{num}/review-frame?page=1&pageTypCd=first&reviewDispYn=Y&isPreview=false&reviewOptDispYn=Y&optSearchBtnAndGraphLayer=Y&reviewBottomBtn=Y&openDetailContents=Y&pageSize=10&isIgnoreAuth=false&lctgrNo=1001312&leafCtgrNo=0&groupProductNo=0&groupFirstViewPrdNo=1717023653&selNo=44919358&prdTypCd=01&myProduct=false&kkukNo=0",

            }
        res = requests.get(base_url, params = req_parmas,  headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
})
        soup = BeautifulSoup(res.text, "html.parser")
        temp = soup.find("div", class_= "c_product_title c_product_title_style3").find("span", class_= "sub").text
        
        tot_num = int(re.findall(r"\d+",temp)[0])
        review_cnt += tot_num
        
        # product with reviews, scrape reviews
        if (tot_num >0) :
            for i in range(1, tot_num//10+2): 
                base_url =f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo={i}&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
                res = requests.get(base_url, params = req_parmas, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'})
                soup = BeautifulSoup(res.text, "html.parser")
                

                for temp in soup.find_all("li", class_="review_list_element"):
                        # customer id
                        try:
                            review_id = temp.find("dt", class_="name").text
                        except:
                            review_id = 0
                        # review date
                        try:
                            review_date = temp.find("span", class_="date").text
                        except:
                            review_date = 0
                        # review_rate
                        try:
                            review_rate = temp.find("p", class_="grade").find("em").text
                        except:
                            review_rate = 0
                        # korean
                        try:
                            review = temp.find("p", class_="cont_review_hide").text
                            review = re.sub(r'\xa0', ' ', review)  # replace \xa0 with space
                            review = re.sub(r'\n', '', review)  # delete \n
                            review_text = review.strip()
                        except:
                            review_text = 0

                        # review_option
                        try:
                            temp_opt = temp.find_all("div", class_="option")[1]
                            review_option = temp_opt.text.replace("선택 옵션", "").strip()
                        except:
                            review_option = 0

                        # review_size
                        try:
                            review_size = temp.find("dl", class_="option_set").find_all("dd")[0].text
                        except:
                            review_size = 0
                        result.append([review_id,review_date,review_rate,review_text,review_option,review_size, num])
        # save result as df and excel
            result_df = pd.DataFrame(result)
            result_df.to_excel(output_filename, index=False)

    # initialize
    result = []
    review_product_num_list = []

    # Merge all review data into one DataFrame
    if os.path.exists(output_filename):
        if os.path.getsize(output_filename) > 0:
            if result_all_df is None:
                result_all_df = pd.read_excel(output_filename)
            else:
                temp_df = pd.read_excel(output_filename)
                result_all_df = pd.concat([result_all_df, temp_df], ignore_index=True)

# Collect and store review data for each product number
for i, file in enumerate(product_numbers_files, start=1):
    with open(file, 'r') as f:
        product_numbers = f.read().splitlines()
    output_filename = f"{output_dir}/df_{i}.xlsx"
    scrape_and_save_reviews(product_numbers, output_filename)
    print(i)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

# 다 되면 다음 셀 실행

In [ ]:
# directory
data_dir = 'data'
output_dir = 'output'
product_numbers_files = [f"{data_dir}/product_numbers_{i}.txt" for i in range(2, 12)]

def scrape_and_save_reviews(product_numbers, output_filename):
    review_cnt = 0
    result = []
    review_product_num_list = []
    path =  r"C:\Users\z1one\OneDrive\바탕 화면\chromedriver-win64\chromedriver.exe"
    s = Service(path)
    # create option to hide widow
    options = webdriver.ChromeOptions()
    options.add_argument("headless")

    driver = webdriver.Chrome(service=s, options  =options)


    for num in product_numbers: # num = productnum
        base_url =f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo=1&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
        req_parmas = {
                "referer":f"https://www.11st.co.kr/products/{num}/review-frame?page=1&pageTypCd=first&reviewDispYn=Y&isPreview=false&reviewOptDispYn=Y&optSearchBtnAndGraphLayer=Y&reviewBottomBtn=Y&openDetailContents=Y&pageSize=10&isIgnoreAuth=false&lctgrNo=1001312&leafCtgrNo=0&groupProductNo=0&groupFirstViewPrdNo=1717023653&selNo=44919358&prdTypCd=01&myProduct=false&kkukNo=0",

            }
        res = requests.get(base_url, params = req_parmas,  headers={'User-Agent':'Mozilla/5.0'})
        soup = BeautifulSoup(res.text, "html.parser")
        temp = soup.find("div", class_= "c_product_title c_product_title_style3").find("span", class_= "sub").text
        
        tot_num = int(re.findall(r"\d+",temp)[0])
        review_cnt += tot_num
        
        # product with reviews, scrape reviews
        if (tot_num >0) :
            for i in range(1, tot_num//10+2): 
                base_url =f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo={i}&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
                res = requests.get(base_url, params = req_parmas)
                soup = BeautifulSoup(res.text, "html.parser")
                

                for temp in soup.find_all("li", class_="review_list_element"):
                        # customer id
                        try:
                            review_id = temp.find("dt", class_="name").text
                        except:
                            review_id = 0
                        # review date
                        try:
                            review_date = temp.find("span", class_="date").text
                        except:
                            review_date = 0
                        # review_rate
                        try:
                            review_rate = temp.find("p", class_="grade").find("em").text
                        except:
                            review_rate = 0
                        # korean
                        try:
                            review = temp.find("p", class_="cont_review_hide").text
                            review = re.sub(r'\xa0', ' ', review)  # replace \xa0 with space
                            review = re.sub(r'\n', '', review)  # delete \n
                            review_text = review.strip()
                        except:
                            review_text = 0

                        # review_option
                        try:
                            temp_opt = temp.find_all("div", class_="option")[1]
                            review_option = temp_opt.text.replace("선택 옵션", "").strip()
                        except:
                            review_option = 0

                        # review_size
                        try:
                            review_size = temp.find("dl", class_="option_set").find_all("dd")[0].text
                        except:
                            review_size = 0
                        result.append([review_id,review_date,review_rate,review_text,review_option,review_size, num])
        # save result as df and excel
            result_df = pd.DataFrame(result)
            result_df.to_excel(output_filename, index=False)

    # initialize
    result = []
    review_product_num_list = []

    # Merge all review data into one DataFrame
    if os.path.exists(output_filename):
        if os.path.getsize(output_filename) > 0:
            if result_all_df is None:
                result_all_df = pd.read_excel(output_filename)
            else:
                temp_df = pd.read_excel(output_filename)
                result_all_df = pd.concat([result_all_df, temp_df], ignore_index=True)

# Collect and store review data for each product number
for i, file in enumerate(product_numbers_files, start=1):
    with open(file, 'r') as f:
        product_numbers = f.read().splitlines()
    output_filename = f"{output_dir}/df_{i}.xlsx"
    scrape_and_save_reviews(product_numbers, output_filename)
    print(i)

In [ ]:
# # 함수: 리뷰 데이터 수집 및 저장
# def scrape_and_save_reviews(product_number, output_filename):
#     result = []
#     review_product_num_list = []
#     path =  r"C:\Users\z1one\OneDrive\바탕 화면\chromedriver-win64\chromedriver.exe"
#     s = Service(path)
#     driver = webdriver.Chrome(service=s)

#     for num in product_number:
#         # 리뷰 데이터 수집 로직 
#         base_url = f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo=1&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0",
#         req_params = {
#                 "referer": f"https://www.11st.co.kr/products/{num}/review-frame?page=1&pageTypCd=first&reviewDispYn=Y&isPreview=false&reviewOptDispYn=Y&optSearchBtnAndGraphLayer=Y&reviewBottomBtn=Y&openDetailContents=Y&pageSize=10&isIgnoreAuth=false&lctgrNo=1001312&leafCtgrNo=0&groupProductNo=0&groupFirstViewPrdNo=1717023653&selNo=44919358&prdTypCd=01&myProduct=false&kkukNo=0",
#             }
#         res = requests.get(base_url, params=req_params)
#         soup = BeautifulSoup(res.text, "html.parser")
#         temp = soup.find("div", class_="c_product_title c_product_title_style3").find("span", class_="sub").text

#         tot_num = int(re.findall(r"\d+", temp)[0])

#         # product with reviews, scrape reviews
#         if tot_num > 0:
#             # if there's a button click
#             base_url = f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo=1&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
#             driver.get(base_url)
#             driver.maximize_window()
#             driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
#             btn_path = '//*[@id="review-list-page-area"]/div[2]/button'
#             driver.find_element(By.XPATH, btn_path).click()

#             for i in range(1, tot_num // 10 + 2):
#                 base_url = f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo={i}&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
#                 res = requests.get(base_url, params=req_params)
#                 soup = BeautifulSoup(res.text, "html.parser")

#                 # 스크랩한 리뷰 데이터 저장
#                 for temp in soup.find_all("li", class_="review_list_element"):
#                     # customer id
#                     try:
#                         review_id = temp.find("dt", class_="name").text
#                     except:
#                         review_id = 0
#                     # review date
#                     try:
#                         review_date = temp.find("span", class_="date").text
#                     except:
#                         review_date = 0
#                     # review_rate
#                     try:
#                         review_rate = temp.find("p", class_="grade").find("em").text
#                     except:
#                         review_rate = 0
#                     # korean
#                     try:
#                         review = temp.find("p", class_="cont_review_hide").text
#                         review = re.sub(r'\xa0', ' ', review)  # replace \xa0 with space
#                         review = re.sub(r'\n', '', review)  # delete \n
#                         review_text = review.strip()
#                     except:
#                         review_text = 0

#                     # review_option
#                     try:
#                         temp_opt = temp.find_all("div", class_="option")[1]
#                         review_option = temp_opt.text.replace("선택 옵션", "").strip()
#                     except:
#                         review_option = 0

#                     # review_size
#                     try:
#                         review_size = temp.find("dl", class_="option_set").find_all("dd")[0].text
#                     except:
#                         review_size = 0

#                     result.append([review_id, review_date, review_rate, review_text, review_option, review_size, num])

#         # 결과를 파일로 저장
#         result_df = pd.DataFrame(result)
#         result_df.to_excel(output_filename, index=False)

#         # 초기화
#         result = []
#         review_product_num_list = []

#         # 모든 리뷰 데이터를 하나의 DataFrame으로 병합
#         if os.path.exists(output_filename):
#             if os.path.getsize(output_filename) > 0:
#                 if result_all_df is None:
#                     result_all_df = pd.read_excel(output_filename)
#                 else:
#                     temp_df = pd.read_excel(output_filename)
#                     result_all_df = pd.concat([result_all_df, temp_df], ignore_index=True)

# # 모든 리뷰 데이터를 저장할 DataFrame 초기화
# result_all_df = None

# # 각 상품 번호에 대한 리뷰 데이터 수집 및 저장
# for i, file in enumerate(product_numbers_files, start=1):
#     with open(file, 'r') as f:
#         product_numbers = f.read().splitlines()
#     output_filename = f"{output_dir}/df_{i}.xlsx"
#     scrape_and_save_reviews(product_numbers, output_filename)

# # result_all_df를 Excel 파일로 저장
# if result_all_df is not None:
#     result_all_df.to_excel(f"{output_dir}/result_all_df.xlsx", index=False)

# print("All Data Merged and Saved as result_all_df.xlsx")

In [ ]:
# [f"{data_dir}/product_numbers_{i}.txt" for i in range(1, 12)]
# for i, file in enumerate(product_numbers_files, start=1):
#     with open(file, 'r') as f:
#         product_numbers = f.read().splitlines()
#     output_filename = f"{output_dir}/df_{i}.xlsx"
#     scrape_and_save_reviews(product_numbers, output_filename)

In [ ]:
# review_cnt = 0
# result= []
# for i, file in enumerate(product_numbers_files, start=1):
#     with open(file, 'r') as f:
#         product_numbers = f.read().splitlines()

# for num in product_numbers: # num = productnum
#     base_url =f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo=1&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
#     req_parmas = {
#             "referer":f"https://www.11st.co.kr/products/{num}/review-frame?page=1&pageTypCd=first&reviewDispYn=Y&isPreview=false&reviewOptDispYn=Y&optSearchBtnAndGraphLayer=Y&reviewBottomBtn=Y&openDetailContents=Y&pageSize=10&isIgnoreAuth=false&lctgrNo=1001312&leafCtgrNo=0&groupProductNo=0&groupFirstViewPrdNo=1717023653&selNo=44919358&prdTypCd=01&myProduct=false&kkukNo=0",

#         }
#     res = requests.get(base_url, params = req_parmas)
#     soup = BeautifulSoup(res.text, "html.parser")
#     temp = soup.find("div", class_= "c_product_title c_product_title_style3").find("span", class_= "sub").text
    
    
#     tot_num = int(re.findall(r"\d+",temp)[0])
#     review_cnt += tot_num
    
#     # 리뷰가 있는 건에 대해 페이지 돌리는 부분
#     if (tot_num >0) :
#         for i in range(1, tot_num//10+2): # 수정한 부분
#             base_url =f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo={i}&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
#             res = requests.get(base_url, params = req_parmas)
#             soup = BeautifulSoup(res.text, "html.parser")
            
#             # 리뷰 모으는 부분
#             for temp in soup.find_all("li", class_= "review_list_element"):                      
#                  # 아이디
#                     try: 
#                         review_id = temp.find("dt", class_= "name").text
#                     except:
#                         review_id = 0
#                     # 리뷰 날짜
#                     try: 
#                         review_date = temp.find("span", class_= "date").text
#                     except:
#                         review_date = 0
#                     # 리뷰 평점
#                     try:
#                         review_rate = temp.find("p", class_= "grade").find("em").text
#                     except: 
#                         review_rate = 0
#                     # 한글 리뷰
#                     import re
#                     try: 
#                         review = temp.find("p", class_="cont_review_hide").text
#                         review = re.sub(r'\xa0', ' ', review)  # \xa0을 스페이스로 대체
#                         review = re.sub(r'\n', '', review)  # \n을 제거
#                         review_text = review.strip()
#                     except:
#                         review_text = 0

#                     # 선택 옵션 
#                     try: 
#                         temp_opt = temp.find_all("div",class_ = "option")[1]
#                         review_option = temp_opt.text.replace("선택 옵션", "").strip()
#                     except: 
#                         review_option = 0

#                     # 사이즈 정보
#                     try: 
#                         review_size = temp.find("dl", class_= "option_set").find_all("dd")[0].text
#                     except:
#                         review_size = 0


#                     result.append([review_id,review_date,review_rate,review_text,review_option,review_size, num])
                   
                    
                
# result_df = pd.DataFrame(result) 
# result_df.to_excel("final_df_3.xlsx")

# print("Done!")

In [ ]:
# result = []
# review_product_num_list = []

# for i, num in enumerate(product_numbers, start=1):  # enumerate를 사용하여 숫자를 추가
#     base_url = f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo=1&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
#     req_params = {
#         "referer": f"https://www.11st.co.kr/products/{num}/review-frame?page=1&pageTypCd=first&reviewDispYn=Y&isPreview=false&reviewOptDispYn=Y&optSearchBtnAndGraphLayer=Y&reviewBottomBtn=Y&openDetailContents=Y&pageSize=10&isIgnoreAuth=false&lctgrNo=1001312&leafCtgrNo=0&groupProductNo=0&groupFirstViewPrdNo=1717023653&selNo=44919358&prdTypCd=01&myProduct=false&kkukNo=0",
#     }
#     res = requests.get(base_url, params=req_params)
#     soup = BeautifulSoup(res.text, "html.parser")
#     temp = soup.find("div", class_="c_product_title c_product_title_style3").find("span", class_="sub").text

#     tot_num = int(re.findall(r"\d+", temp)[0])

#     # product with reviews, scrape reviews
#     if tot_num > 0:
#         # if there's a button click
#         base_url = f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo=1&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
#         driver.get(base_url)
#         driver.maximize_window()
#         driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
#         btn_path = '//*[@id="review-list-page-area"]/div[2]/button'
#         driver.find_element(By.XPATH, btn_path).click()

#         for i in range(1, tot_num // 10 + 2):
#             base_url = f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo={i}&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
#             res = requests.get(base_url, params=req_params)
#             soup = BeautifulSoup(res.text, "html.parser")

#             # 스크랩한 리뷰 데이터 저장
#             for temp in soup.find_all("li", class_="review_list_element"):
#                 # customer id
#                 try:
#                     review_id = temp.find("dt", class_="name").text
#                 except:
#                     review_id = 0
#                 # review date
#                 try:
#                     review_date = temp.find("span", class_="date").text
#                 except:
#                     review_date = 0
#                 # review_rate
#                 try:
#                     review_rate = temp.find("p", class_="grade").find("em").text
#                 except:
#                     review_rate = 0
#                 # korean
#                 try:
#                     review = temp.find("p", class_="cont_review_hide").text
#                     review = re.sub(r'\xa0', ' ', review)  # replace \xa0 with space
#                     review = re.sub(r'\n', '', review)  # delete \n
#                     review_text = review.strip()
#                 except:
#                     review_text = 0

#                 # review_option
#                 try:
#                     temp_opt = temp.find_all("div", class_="option")[1]
#                     review_option = temp_opt.text.replace("선택 옵션", "").strip()
#                 except:
#                     review_option = 0

#                 # review_size
#                 try:
#                     review_size = temp.find("dl", class_="option_set").find_all("dd")[0].text
#                 except:
#                     review_size = 0

#                 result.append([review_id, review_date, review_rate, review_text, review_option, review_size, num])
#                 review_product_num_list.append([num, id_review_cnt])

#     # 결과를 파일로 저장 (파일 이름에 숫자 추가)
#     result_df = pd.DataFrame(result)
#     result_df.to_excel(f"df_{i}.xlsx", index=False)

#     review_product_num_list_df = pd.DataFrame(review_product_num_list)
#     review_product_num_list_df.to_excel(f"list_{i}.xlsx", index=False)

#     result = []  # 결과 초기화
#     review_product_num_list = []  # 리스트 초기화

# print("Done!")


In [ ]:

# # 빈 DataFrame 생성
# result_all_df = pd.DataFrame(columns=["Customer ID", "Review Date", "Review Rate", "Review Text", "Review Option", "Review Size", "Product Number"])

# # 각 df 파일을 읽어와 병합
# for i in range(1, 12):  # 파일 수에 따라 범위 조절 (1부터 11까지)
#     file_name = f"df_{i}.xlsx"
#     df = pd.read_excel(file_name, index_col=0)
#     result_all_df = pd.concat([result_all_df, df], ignore_index=True)

# # 병합된 DataFrame을 하나의 Excel 파일로 저장
# result_all_df.to_excel("result_all_df.xlsx", index=False)

# print("All Data Merged and Saved as result_all_df.xlsx")


In [ ]:
# # num = 5501668011

# # if there's a button, click by selenium. Otherwise, scrape data 

# product_num_list = ['3582216563', '3581969180', '3575728745', '3553883972', '3553773267', '3548619969', '3547722066', '3538668940', '5749296973', '5736750071', '5734222597', '5730209511', '5727478506', '5724948577', '5724093100', '5721562544', '5718004842', '5713005354', '5712891127', '5710181243', '5709558715', '5706354212', '5705711313', '5702525032', '5690431293', '5686143338', '5682303672', '5682151133', '5681376359', '5663709514', '5663337278', '5662789155', '5661810504', '5647350718', '5645900049', '5645263952', '5643503235', '5632378103', '5628970761', '5608429523', '5601040035', '5593070811', '5582168281', '5580332878', '5575242450', '5571587220', '5568539325', '5560769703', '5557923041', '5555079652', '5552034477', '5551956038', '5551887783', '5550391662', '5549340987', '5548984301', '5548942666', '5548504097', '5548223301', '5546765257', '5546404656', '5546351791', '5544494220', '5543955070', '5543127192', '5541456702', '5540774677', '5540721839', '5540498884', '5540333989', '5540193784', '5539758737', '5539492248', '5539165026', '5537376806', '5537348007', '5537087058', '5536658636', '5536168116', '5535807311', '5534529663', '5534151756', '5533889665', '5532675757', '5532310394', '5532294502', '5529098186', '5527825277', '5524167392', '5511392395', '5501668011', '5456386311', '5440430637', '5428179751', '5419128309', '5384523822', '5352369554', '5351971255', '5282000509', '5248705272', '5226162096', '5226091695', '5209693816', '5191096353', '5166400682', '5116795751', '5073087508', '5054357353', '4958845288', '4911938299', '4843125819', '4812183210', '4797994063', '4742853021', '4703366249', '4697194978', '4673615555', '4651107738', '4519249467', '4474093988', '4465828899', '4461303463', '4430906060', '4419233746', '4418745988', '4417269575', '4415431195', '4415324010', '4415317334', '4329864647', '4311192497', '4303286103', '4272897183', '4240473895', '4030360740', '3980154902', '3943416786', '3921300976', '3874704817', '3850694187', '3843099414', '3780784961', '3772067959', '3738968489', '3714019020', '3694147974', '3643006671']


# result = []
# # count the number of review pages
# for num in product_num_list: # num = productnum
#     base_url =f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo=1&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
#     req_parmas = {
#             "referer":f"https://www.11st.co.kr/products/{num}/review-frame?page=1&pageTypCd=first&reviewDispYn=Y&isPreview=false&reviewOptDispYn=Y&optSearchBtnAndGraphLayer=Y&reviewBottomBtn=Y&openDetailContents=Y&pageSize=10&isIgnoreAuth=false&lctgrNo=1001312&leafCtgrNo=0&groupProductNo=0&groupFirstViewPrdNo=1717023653&selNo=44919358&prdTypCd=01&myProduct=false&kkukNo=0",

#         }
#     res = requests.get(base_url, params = req_parmas)
#     soup = BeautifulSoup(res.text, "html.parser")
#     temp = soup.find("div", class_= "c_product_title c_product_title_style3").find("span", class_= "sub").text
    
#     tot_num = int(re.findall(r"\d+",temp)[0])
    
#     # product with reviews, scrape reviews 
#     if (tot_num >0) :
#         # if there's a button click 
#         base_url =f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo=1&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
#         driver.get(base_url)
#         driver.maximize_window()        
#         driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")        
#         btn_path = '//*[@id="review-list-page-area"]/div[2]/button'
#         driver.find_element(By.XPATH,btn_path).click()
        
        
#         for i in range(1, tot_num//10+2): 
#             base_url =f"https://www.11st.co.kr/products/{num}/review-list??pageSize=10&pageNo={i}&myProduct=false&pntVals=&rtype=&sortType=01&themeNm=&optNm=&kkukNo=0"
#             res = requests.get(base_url, params = req_parmas)
#             soup = BeautifulSoup(res.text, "html.parser")
            
#             # scraping review data
#             for temp in soup.find_all("li", class_= "review_list_element"):                      
#                  # customer id
#                     try: 
#                         review_id = temp.find("dt", class_= "name").text
#                     except:
#                         review_id = 0
#                     # review date
#                     try: 
#                         review_date = temp.find("span", class_= "date").text
#                     except:
#                         review_date = 0
#                     # review_rate
#                     try:
#                         review_rate = temp.find("p", class_= "grade").find("em").text
#                     except: 
#                         review_rate = 0
#                     # korean 
#                     try: 
#                         review = temp.find("p", class_="cont_review_hide").text
#                         review = re.sub(r'\xa0', ' ', review)  # replace \xa0 with space
#                         review = re.sub(r'\n', '', review)  # delete \n
#                         review_text = review.strip()
#                     except:
#                         review_text = 0

#                     # review_option
#                     try: 
#                         temp_opt = temp.find_all("div",class_ = "option")[1]
#                         review_option = temp_opt.text.replace("선택 옵션", "").strip()
#                     except: 
#                         review_option = 0

#                     # review_size
#                     try: 
#                         review_size = temp.find("dl", class_= "option_set").find_all("dd")[0].text
#                     except:
#                         review_size = 0


#                     result.append([review_id,review_date,review_rate,review_text,review_option,review_size, num])
                    
                    
                
# result_df = pd.DataFrame(result) 
# result_df.to_excel("df_1.xlsx")

# review_product_num_list_df = pd.DataFrame(review_product_num_list)
# review_product_num_list_df.to_excel("list_1.xlsx")

# print("Done!")